In [2]:
import random
import h5py as h5
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [3]:
file = h5.File('output_digi_HDF_Mg22_Ne20pp_8MeV.h5', 'r')
#print(file["Event_[3]"][:][:])

In [4]:
sample_size = 128
keys = list(file.keys())
length = len(keys)

In [5]:
event_lens = np.zeros(length,int)
for i in range (length):
    event = keys[i]
    event_lens[i] = len(file[event]) 

In [6]:
event_data = np.zeros((length, np.max(event_lens), 12), float)
for n in tqdm.tqdm(range(len(keys))):
    name = keys[n]
    event = file[name]
    ev_len = len(event)
    #converting event into an array
    for i,e in enumerate(event):
        instant = np.array(list(e))
        event_data[n][i][:] = np.array(instant)    
np.save('Mg22_Ne20pp', event_data)

100%|██████████| 10000/10000 [10:36<00:00, 15.71it/s]


In [7]:
data = np.load('Mg22_Ne20pp.npy')
file_name = 'Mg22_size_' + str(sample_size) + '.h5'
new = h5.File(file_name, 'w')
for n in tqdm.tqdm(range(length)):
    name = keys[n]
    ev_lens = event_lens[n]
    new_event = np.zeros((sample_size, 13), float)

    #making new array for normalized event
    if ev_lens == sample_size:
        new_event[:,:-1] = data[n,:ev_lens,:]
    else: 
        for i in range(sample_size):
            row = random.randint(0, ev_lens - 1)
            new_event[i,:-1] = data[n,row,:] 
    unique_targets = np.unique(data[n,:ev_lens,5]) #get total number of tracks in each event 
    new_event[0,-1]= unique_targets.size - 1 #append track id to end of first instance in event
    
    new.create_dataset(name, data = new_event)

100%|██████████| 10000/10000 [00:06<00:00, 1437.81it/s]


In [8]:
#NOT completely random sampling!
data = np.load('Mg22_Ne20pp.npy')
file_name = 'Mg22_size' + str(sample_size) + '.h5'
new = h5.File(file_name, 'w')
for i in tqdm.tqdm(range(length)):
    ev_len = event_lens[i]    #length of event-- i.e. number of instances
    particle_ids = data[i,:event_lens[i],5]
    label, distr = np.unique(particle_ids, return_counts=True)
    shortest = label[np.argmin(distr)]
    shortest_ind = np.argwhere(particle_ids == shortest)
    new_event = np.zeros((sample_size, 13), float)    #empty array for sampled event data
    if ev_len == sample_size:    #if array is already preferred length
        new_event[:,:-1] = data[i,:ev_len,:]
    else:
        for n in range(sample_size):    #the first instances sampled will be those belonging to the shortest track
            if n < shortest.size:
                new_event[n,:-1] = data[i,shortest_ind[n],:]
            else:
                row = random.randint(0, ev_len - 1)
                new_event[n,:-1] = data[i,row,:]
    unique_point_ids = np.unique(data[i,:ev_len,5])    #array of unique particle IDs
    new_event[0,-1] = unique_point_ids.size - 1    #number of unique particles, scaled to start at 0
    new.create_dataset(keys[i], data = new_event)    #creating new dataset within the h5 file for the event

100%|██████████| 10000/10000 [00:08<00:00, 1206.15it/s]


In [9]:
print(event[60])
# sixth number is particle id

(2.45410442, 9.95206547, 504., 195, 5.83169867, 4, 5, 0., 1.62471315e-05, 0., 1, 1)


In [10]:
#creating data sets with fewer beam events and no 2, 5, or 6 track events
name = 'Mg22_size128'
data = h5.File(name + '.h5', 'r')
keys = list(data.keys())
length = len(keys)
new = h5.File(name + '_edited.h5', 'w')
for i in tqdm.tqdm(range(length)):
    event = data[keys[i]]
    unique_point_ids = np.unique(data[keys[i]][:event_lens[i],5])    #array of unique particle IDs
    new_event[0,-1] = unique_point_ids.size - 1 
    num_tracks = new_event[0,-1]
    #omitting the latter half of beam events along with 2, 5, and 6 track events
    if (num_tracks == 0 and i > 5000) or num_tracks == 1 or num_tracks == 4 or num_tracks == 5:
        continue
    else:
        new.create_dataset(keys[i], data = event)

100%|██████████| 10000/10000 [00:09<00:00, 1110.50it/s]


In [11]:
#checking the range of number of tracks and the distribution in the original data
data = np.load('Mg22_Ne20pp.npy')
num_tracks = np.zeros(len(keys),int)
for i in range(data.shape[0]):
    unique_point_ids = np.unique(data[i,:event_lens[i],5])    #array of unqiue particles IDs
    num_tracks[i] = unique_point_ids.size - 1
label, og_distr = np.unique(num_tracks, return_counts=True)
print(og_distr)

[5002  104 2554 2319   16    5]


In [12]:
#cheking the number of tracks and distribution in the processed data
data = h5.File('Mg22_size128_edited.h5', 'r')
keys = list(data.keys())
num_tracks = np.zeros(len(keys),int)
for i in range(len(keys)):
    event = data[keys[i]]
    unique_point_ids = np.unique(event[:,5])    #array of unqiue particles IDs
    num_tracks[i] = unique_point_ids.size - 1
label, new_distr = np.unique(num_tracks, return_counts=True)
print(new_distr)
print(np.sum(np.abs(new_distr - og_distr))//2)

[2503 2550 2308]


ValueError: operands could not be broadcast together with shapes (3,) (6,) 

In [13]:
#setting aside a test set from the all events dataset
name = 'Mg22_size' + str(sample_size)
whole = h5.File(name + '.h5','r')
test = h5.File(name + '_test.h5', 'w')
rest = h5.File(name + '_rest.h5', 'w')
keys = list(whole.keys())
length = len(keys)
test_len = int(0.2*length)
test_set_indices = np.random.choice(range(length), test_len, replace=False)
for i in tqdm.tqdm(range(length)):
    event = whole[keys[i]]
    if np.isin(i, test_set_indices, assume_unique=True):
        test.create_dataset(keys[i], data = event)
    else:
        rest.create_dataset(keys[i], data = event)

100%|██████████| 10000/10000 [00:09<00:00, 1073.87it/s]


In [14]:
#splits remaining events data into training and validation sets
name = 'Mg22_size' + str(sample_size)
whole = h5.File(name + '_rest.h5','r')
train = h5.File(name + '_train.h5', 'w')
val = h5.File(name + '_val.h5', 'w')
keys = list(whole.keys())
length = len(keys)
val_len = int(0.25*length)     #20% of 80%
val_set_indices = np.random.choice(range(length), val_len, replace=False)
for i in tqdm.tqdm(range(length)):
    event = whole[keys[i]]
    if np.isin(i, val_set_indices, assume_unique=True):
        val.create_dataset(keys[i], data = event)
    else:
        train.create_dataset(keys[i], data = event)

100%|██████████| 8000/8000 [00:07<00:00, 1057.41it/s]


In [3]:
files = h5.File('Mg22_size128_train.h5', 'r')
keys = list(files.keys())
print(files[keys[1]][1])

[-7.36236525e+00 -1.45075214e+00  8.75200000e+02  3.11000000e+02
  2.39268389e+02  0.00000000e+00  3.00000000e+00  0.00000000e+00
  1.53025219e-04  0.00000000e+00  2.20000000e+01  1.20000000e+01
  0.00000000e+00]
